In [1]:
%%bash

pip install -q kaggle
mkdir -p ~/.kaggle
cp drive/MyDrive/kaggle.json ~/.kaggle/
chmod 600 /root/.kaggle/kaggle.json
kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection

100%|██████████| 15.1M/15.1M [00:02<00:00, 6.10MB/s]


In [ ]:
!unzip brain-mri-images-for-brain-tumor-detection.zip

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras

In [9]:
data_gen = ImageDataGenerator(
    zoom_range=0.2,
    shear_range=0.2,
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True
)

In [10]:
train_set = data_gen.flow_from_directory(
    "brain_tumor_dataset/",
    target_size=(224, 224),
    class_mode='binary',
    seed=123,
    subset='training',
    batch_size=32
)

valid_set = data_gen.flow_from_directory(
    "brain_tumor_dataset/",
    target_size=(224, 224),
    class_mode='binary',
    seed=123,
    subset='validation',
    batch_size=32
)

Found 203 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [15]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32, kernel_size=3, input_shape=(224, 224, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=3))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=3))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(420, activation='relu'))
model.add(keras.layers.Dense(120, activation='relu')) 
model.add(keras.layers.Dense(60, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [16]:
model.fit(x=train_set, epochs=20, batch_size=32, validation_data=valid_set, callbacks=[early_stopping])

Epoch 1/20
7/7 [==============================] - 4s 591ms/step - loss: 0.6485 - accuracy: 0.6552 - val_loss: 0.6695 - val_accuracy: 0.6600
Epoch 2/20
7/7 [==============================] - 3s 461ms/step - loss: 0.6333 - accuracy: 0.6256 - val_loss: 0.6387 - val_accuracy: 0.6800
Epoch 3/20
7/7 [==============================] - 3s 454ms/step - loss: 0.6062 - accuracy: 0.6798 - val_loss: 0.6254 - val_accuracy: 0.6600
Epoch 4/20
7/7 [==============================] - 3s 443ms/step - loss: 0.5697 - accuracy: 0.7586 - val_loss: 0.6288 - val_accuracy: 0.7200
Epoch 5/20
7/7 [==============================] - 3s 503ms/step - loss: 0.5908 - accuracy: 0.6749 - val_loss: 0.6213 - val_accuracy: 0.7200
Epoch 6/20
7/7 [==============================] - 3s 445ms/step - loss: 0.5760 - accuracy: 0.6946 - val_loss: 0.5985 - val_accuracy: 0.7200
Epoch 7/20
7/7 [==============================] - 3s 487ms/step - loss: 0.5949 - accuracy: 0.7094 - val_loss: 0.6159 - val_accuracy: 0.7400
Epoch 8/20
7/7 [====